In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import streamlit as st
from pycaret.regression import *

LABEL_ENCODER = None

def remove_null(data):
    data.dropna(inplace=True)
    return data

def convert_dates_to_one_format(data):
    data['Travel Date'] = pd.to_datetime(data['Travel Date'], errors='coerce')
    data['Travel Date'] = data['Travel Date'].dt.strftime('%Y-%m-%d')
    return data

def replace_space_with_underscore(name):
    return name.replace(' ', '_')

def create_label_encoding(data):
    global LABEL_ENCODER
    cat_cols = data.select_dtypes(include=['object']).columns
    LABEL_ENCODER = LabelEncoder()
    for col in cat_cols:
        data[col] = LABEL_ENCODER.fit_transform(data[col])
        np.save('classes_{0}.npy'.format(replace_space_with_underscore(col)), LABEL_ENCODER.classes_, allow_pickle=True)

def change_int32_to_int64(data):
    for col in data.columns:
        if data[col].dtype == 'int64':
            data[col] = data[col].astype('int32')

def data_preprocessing(data):
    data.drop(['Uniq Id'], axis=1, inplace=True)
    remove_null(data)
    change_int32_to_int64(data)
    convert_dates_to_one_format(data)
    create_label_encoding(data)
    
### preprocessing 

def split_category(value):
    vals = []
    if '|' in value:
        vals = value.split('|')
    else:
        vals.append(value)
    return vals

def split_columns_with_multiple_values(data):
    data['Airline'] = data['Airline'].apply(split_category)
    data['Destination'] = data['Destination'].apply(split_category)
    data['Places Covered'] = data['Places Covered'].apply(split_category)
    data['Sightseeing Places Covered'] = data['Sightseeing Places Covered'].apply(split_category)
    return data

def one_hot_encoding_on_columns(data):
    dummy_type = pd.get_dummies(data['Package Type'], prefix='type')
    data.drop(columns=['Package Type'], inplace=True)
    data = pd.concat([data, dummy_type], axis=1)

    dummy_city = pd.get_dummies(data['Start City'], prefix='sc')
    data.drop(columns=['Start City'], inplace=True)
    data = pd.concat([data, dummy_city], axis=1)

    label_encoder2 = LabelEncoder().fit(data['Cancellation Rules'])
    data['Cancellation Rules'] = label_encoder2.transform(data['Cancellation Rules'])
    return data


def show_category(series):
    values = {}
    for val in series:
        for each in val:
            if each in values:
                values[each] += 1
            else:
                values[each] = 1
    return values

def make_feature_col(series, all_keys):
    feature_dict = {}
    for key in all_keys:
        feature_dict[key] = []

    for items in series:
        for key in all_keys:
            if key not in items:
                feature_dict[key].append(0)
            else:
                feature_dict[key].append(1)

    return pd.DataFrame(feature_dict)

def structuring_columns(data):
    change = lambda pc: [each+'_ae' for each in pc]
    A_all_keys = show_category(data['Airline']).keys()
    A_all_keys = change(A_all_keys)
    airline = make_feature_col(data['Airline'], A_all_keys)
    data = pd.concat([data, airline], axis=1)

    change = lambda pc: [each+'_ds' for each in pc]
    D_all_keys = show_category(data['Destination']).keys()
    D_all_keys = change(D_all_keys)
    destination = make_feature_col(data['Destination'], D_all_keys)
    data = pd.concat([data, destination], axis=1)

    change = lambda pc: [each+'_pc' for each in pc]
    PC_all_keys = show_category(data['Places Covered']).keys()
    PC_all_keys = change(PC_all_keys)
    p_covered = make_feature_col(data['Places Covered'], PC_all_keys)
    data = pd.concat([data, p_covered], axis=1)

    return data

def remove_useless_columns(data):
    data.drop('Uniq Id', inplace=True, axis=1)
    data.drop('Package Name', inplace=True, axis=1)

    return data
  
def preprocess_data(data):
    data = remove_useless_columns(data)
    data = split_columns_with_multiple_values(data)
    data = one_hot_encoding_on_columns(data)
    data = structuring_columns(data)
    return(data)
    
 ### splitting data 
 
def split_data(data):
    features = ['Flight Stops', 'Meals', 'Cancellation Rules']
    other_features = data.columns[13:].to_list()
    features.extend(other_features)
    target = ['Per Person Price']

    X = data[features]
    y = data[target]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

    print('Shape of train set:', X_train.shape)
    print('Shape of test set:', X_test.shape)

    return X_train, X_test, y_train, y_test

train = pd.read_csv('dataset\Train.csv')
validation = pd.read_csv('dataset\Test.csv')

final_train_data = preprocess_data(train)
final_validation_data = preprocess_data(validation)

In [16]:
print('================TRAINING DATA==================')
display(final_train_data)
print(final_train_data.shape)

print('================VALIDATION DATA==================')
display(final_validation_data)
print(final_validation_data.shape)

,Destination,Itinerary,Places Covered,Travel Date,Hotel Details,Airline,Flight Stops,Meals,Sightseeing Places Covered,Cancellation Rules,Per Person Price,type_Budget,type_Deluxe,type_Luxury,type_Premium,type_Standard,sc_Mumbai,sc_New Delhi,Not Available_ae,IndiGo_ae,Spicejet_ae,Go Air_ae,Air India_ae,Emirates_ae,Vistara_ae,Thai Airways_ae,AirAsia Indonesia_ae,Srilankan Airlines_ae,AirAsia_ae,Air India Express_ae,Malindo Air_ae,Malaysia Airlines_ae,Qantas Airways_ae,Air New Zealand_ae,Singapore Airlines_ae,Air Seychelles_ae,Etihad Airways_ae,AirAsia X_ae,Scoot_ae,Air Mauritius_ae,Oman Air_ae,Thai AirAsia_ae,Jetstar Asia_ae,Saudia_ae,Thai Vietjet Air_ae,EgyptAir_ae,Garuda Indonesia_ae,Silkair_ae,Bangkok Airways_ae,Kenya Airways_ae,Aeroflot_ae,Philippine Airlines_ae,Jetstar Airways_ae,Swiss_ae,Gulf Air_ae,Lufthansa_ae,Air Baltic_ae,Uzbekistan Airways_ae,Kuwait Airways_ae,China Southern_ae,Finnair_ae,Qatar Airways_ae,Airconnect_ae,China Eastern_ae,Norwegian_ae,British Airways_ae,Virgin Atlantic_ae,Turkish Airlines_ae,Brussels Airlines_ae,Cathay Pacific_ae,Air Austral_ae,South African Airways_ae,Ethiopian Airlines_ae,Sichuan Airlines_ae,New Delhi_ds,Shimla_ds,Manali_ds,Chandigarh_ds,Srinagar_ds,Pahalgam_ds,Udaipur_ds,Chittorgarh_ds,Munnar_ds,Kumarakom_ds,Allepey_ds,Kovalam and Poovar_ds,Pattaya_ds,Bangkok_ds,Cochin_ds,Jaipur_ds,Kasol_ds,Gulmarg_ds,Kanyakumari_ds,Guwahati_ds,Mount Abu_ds,Nainital_ds,Kausani_ds,Corbett_ds,Kodaikanal_ds,Dharamshala_ds,Ooty_ds,Mysore_ds,Agra_ds,Dubai_ds,Goa_ds,Gangtok_ds,Pelling_ds,Darjeeling_ds,Dalhousie_ds,Amritsar_ds,Coorg_ds,Bandipur_ds,Thekkady_ds,Lachung_ds,Bangalore_ds,Kandy_ds,Bentota_ds,Colombo_ds,Parwanoo_ds,Nameri_ds,Wayanad_ds,Kalimpong_ds,Bali_ds,Singapore_ds,Kovalam_ds,Negombo_ds,Nuwara Eliya_ds,Madurai_ds,Rameshwaram_ds,Leh_ds,Kullu_ds,Jodhpur_ds,Jaisalmer_ds,Port Blair_ds,Havelock_ds,Shillong_ds,Namchi_ds,Pondicherry_ds,Mahabalipuram_ds,Katra_ds,Mussoorie_ds,Abu Dhabi_ds,Rishikesh_ds,Kuala Lumpur_ds,Aurangabad_ds,Shirdi_ds,Kasauli_ds,Queenstown_ds,Kumbalgarh_ds,Ranikhet_ds,Niel Island_ds,Tirupati_ds,Kaziranga_ds,Ranthambhore_ds,Ajmer_ds,Mathura_ds,Haridwar_ds,Bikaner_ds,Auckland_ds,Rotorua_ds,Christchurch_ds,Sharjah_ds,Mauritius_ds,Praslin Island_ds,Mahe Island_ds,Poovar_ds,Varkala_ds,Pushkar_ds,Langkawi_ds,Yala_ds,Thimpu_ds,Paro_ds,Chail_ds,Tanjore_ds,Kumbakonam_ds,Tiruchirapally_ds,Patnitop_ds,Phuket_ds,Krabi town_ds,Siliguri_ds,Almora_ds,Punakha_ds,Pali_ds,Mumbai_ds,Alibagh_ds,Ganpatipule_ds,Lonavala And Khandala_ds,Sigiriya_ds,Galle_ds,Genting_ds,Bharatpur_ds,Cherrapunjee_ds,Palampur_ds,Ras Al Khaimah_ds,Penang_ds,Paris_ds,Bintan Island_ds,Nubra Valley_ds,Chintpurni_ds,Kangra_ds,Kabini_ds,Batam_ds,Lachen_ds,Dambulla_ds,Kuta_ds,Seminyak_ds,Nasik_ds,Saputara_ds,Jammu_ds,Ahmedabad_ds,Jamnagar_ds,Dwarka_ds,Somnath_ds,Anuradhapura_ds,Lucknow_ds,Varanasi_ds,Jakarta_ds,Johor Bahru_ds,Trivandrum_ds,Ranakpur_ds,Trincomalee_ds,Sonmarg_ds,Chennai_ds,Kanchipuram_ds,Nanded_ds,Ahmednagar_ds,Gokarna_ds,Melbourne_ds,Gold Coast_ds,Cairns_ds,Sydney_ds,Nawalgarh_ds,Bundi_ds,Prague_ds,Phuentsholing_ds,Moscow_ds,Saint Petersburg_ds,Hassan_ds,Hampi_ds,Franz Josef_ds,Kota Kinabalu_ds,Chikmangalur_ds,Kishangarh_ds,Mandawa_ds,Singapore Cruise_ds,Hong Kong_ds,Vadodara_ds,Joshimath_ds,Copenhagen_ds,Daman_ds,Helsinki_ds,Gwalior_ds,Orchha_ds,Khajuraho_ds,Alwar_ds,Oslo_ds,New Delhi_pc,Shimla_pc,Manali_pc,Chandigarh_pc,Srinagar_pc,Pahalgam_pc,Udaipur_pc,Chittorgarh_pc,Munnar_pc,Kumarakom_pc,Allepey_pc,Kovalam and Poovar_pc,Pattaya_pc,Bangkok_pc,Cochin_pc,Jaipur_pc,Kasol_pc,Gulmarg_pc,Kanyakumari_pc,Guwahati_pc,Mount Abu_pc,Nainital_pc,Kausani_pc,Corbett_pc,Kodaikanal_pc,Dharamshala_pc,Ooty_pc,Mysore_pc,Agra_pc,Dubai_pc,Goa_pc,Gangtok_pc,Pelling_pc,Darjeeling_pc,Dalhousie_pc,Amritsar_pc,Coorg_pc,Bandipur_pc,Thekkady_pc,Lachung_pc,Bangalore_pc,Kandy_pc,Bentota_pc,Colombo_pc,Parwanoo_pc,Nameri_pc,Wayanad_pc,Kalimpong_pc,Bali_pc,Singapore_pc,Kovalam_pc,Negombo_pc,Nuwara Eliya_pc,Madurai_pc,Rameshwaram_pc,Leh_pc,Kullu

(21000, 438)


,Destination,Itinerary,Places Covered,Travel Date,Hotel Details,Airline,Flight Stops,Meals,Sightseeing Places Covered,Cancellation Rules,type_Budget,type_Deluxe,type_Luxury,type_Premium,type_Standard,sc_Mumbai,sc_New Delhi,IndiGo_ae,Not Available_ae,Emirates_ae,Spicejet_ae,Air India_ae,Lufthansa_ae,Srilankan Airlines_ae,Vistara_ae,Go Air_ae,AirAsia X_ae,Malindo Air_ae,Bangkok Airways_ae,AirAsia Indonesia_ae,AirAsia_ae,Malaysia Airlines_ae,Jetstar Asia_ae,Oman Air_ae,Air India Express_ae,Singapore Airlines_ae,Silkair_ae,Thai Airways_ae,Thai AirAsia_ae,Cathay Pacific_ae,Qantas Airways_ae,Air Mauritius_ae,Scoot_ae,EgyptAir_ae,Air New Zealand_ae,Air Austral_ae,Saudia_ae,Aeroflot_ae,Air Seychelles_ae,Etihad Airways_ae,Swiss_ae,Air Baltic_ae,Turkish Airlines_ae,Thai Vietjet Air_ae,Qatar Airways_ae,Ethiopian Airlines_ae,Garuda Indonesia_ae,Goa_ds,Bandipur_ds,Munnar_ds,Cochin_ds,Thekkady_ds,Allepey_ds,Mauritius_ds,Kaziranga_ds,Guwahati_ds,Gangtok_ds,Lachen_ds,Lachung_ds,Paris_ds,Coorg_ds,Ooty_ds,Sharjah_ds,Kuala Lumpur_ds,Johor Bahru_ds,Amritsar_ds,Mysore_ds,Dharamshala_ds,Dalhousie_ds,Darjeeling_ds,Shimla_ds,Manali_ds,Chandigarh_ds,Bangalore_ds,New Delhi_ds,Agra_ds,Jaipur_ds,Kovalam and Poovar_ds,Bali_ds,Phuket_ds,Krabi town_ds,Pattaya_ds,Bangkok_ds,Colombo_ds,Kandy_ds,Madurai_ds,Kodaikanal_ds,Rameshwaram_ds,Kanyakumari_ds,Kovalam_ds,Udaipur_ds,Mount Abu_ds,Port Blair_ds,Havelock_ds,Jodhpur_ds,Abu Dhabi_ds,Dubai_ds,Singapore_ds,Jaisalmer_ds,Kabini_ds,Pelling_ds,Chittorgarh_ds,Shillong_ds,Haridwar_ds,Kasauli_ds,Srinagar_ds,Gulmarg_ds,Pahalgam_ds,Chail_ds,Nainital_ds,Kausani_ds,Ranikhet_ds,Corbett_ds,Kumarakom_ds,Tirupati_ds,Kalimpong_ds,Bentota_ds,Galle_ds,Bikaner_ds,Katra_ds,Leh_ds,Pondicherry_ds,Anuradhapura_ds,Sigiriya_ds,Nuwara Eliya_ds,Yala_ds,Hassan_ds,Hampi_ds,Nasik_ds,Saputara_ds,Cherrapunjee_ds,Mathura_ds,Thimpu_ds,Paro_ds,Kullu_ds,Bintan Island_ds,Negombo_ds,Pushkar_ds,Shirdi_ds,Trivandrum_ds,Ajmer_ds,Langkawi_ds,Mussoorie_ds,Parwanoo_ds,Varkala_ds,Genting_ds,Niel Island_ds,Chikmangalur_ds,Rishikesh_ds,Wayanad_ds,Ranthambhore_ds,Phuentsholing_ds,Punakha_ds,Kanchipuram_ds,Palampur_ds,Auckland_ds,Queenstown_ds,Patnitop_ds,Kuta_ds,Seminyak_ds,Mahabalipuram_ds,Mandawa_ds,Kumbakonam_ds,Tanjore_ds,Tiruchirapally_ds,Nubra Valley_ds,Aurangabad_ds,Ahmednagar_ds,Lonavala And Khandala_ds,Bharatpur_ds,Christchurch_ds,Trincomalee_ds,Kasol_ds,Batam_ds,Almora_ds,Poovar_ds,Namchi_ds,Gwalior_ds,Orchha_ds,Khajuraho_ds,Jammu_ds,Ranakpur_ds,Kumbalgarh_ds,Nanded_ds,Ahmedabad_ds,Sonmarg_ds,Kota Kinabalu_ds,Siliguri_ds,Kishangarh_ds,Chintpurni_ds,Kangra_ds,Praslin Island_ds,Mahe Island_ds,Chennai_ds,Hong Kong_ds,Mumbai_ds,Gokarna_ds,Lucknow_ds,Varanasi_ds,Prague_ds,Jamnagar_ds,Dwarka_ds,Somnath_ds,Nameri_ds,Jakarta_ds,Dambulla_ds,Ganpatipule_ds,Moscow_ds,Saint Petersburg_ds,Alwar_ds,Bundi_ds,Copenhagen_ds,Vadodara_ds,Pali_ds,Joshimath_ds,Nawalgarh_ds,Daman_ds,Ras Al Khaimah_ds,Franz Josef_ds,Rotorua_ds,Penang_ds,Melbourne_ds,Gold Coast_ds,Cairns_ds,Sydney_ds,Alibagh_ds,Goa_pc,Bandipur_pc,Munnar_pc,Cochin_pc,Thekkady_pc,Allepey_pc,Mauritius_pc,Kaziranga_pc,Guwahati_pc,Gangtok_pc,Lachen_pc,Lachung_pc,Paris_pc,Coorg_pc,Ooty_pc,Sharjah_pc,Kuala Lumpur_pc,Johor Bahru_pc,Amritsar_pc,Mysore_pc,Dharamshala_pc,Dalhousie_pc,Darjeeling_pc,Shimla_pc,Manali_pc,Chandigarh_pc,Bangalore_pc,New Delhi_pc,Agra_pc,Jaipur_pc,Kovalam and Poovar_pc,Bali_pc,Phuket_pc,Krabi town_pc,Pattaya_pc,Bangkok_pc,Colombo_pc,Kandy_pc,Madurai_pc,Kodaikanal_pc,Rameshwaram_pc,Kanyakumari_pc,Kovalam_pc,Udaipur_pc,Mount Abu_pc,Port Blair_pc,Havelock_pc,Jodhpur_pc,Abu Dhabi_pc,Dubai_pc,Singapore_pc,Jaisalmer_pc,Kabini_pc,Pelling_pc,Chittorgarh_pc,Shillong_pc,Haridwar_pc,Kasauli_pc,Srinagar_pc,Gulmarg_pc,Pahalgam_pc,Chail_pc,Nainital_pc,Kausani_pc,Ranikhet_pc,Corbett_pc,Kumarakom_pc,Tirupati_pc,Kalimpong_pc,Bentota_pc,Galle_pc,Bikaner_pc,Katra_pc,Leh_pc,Pondicherry_pc,Anuradhapura_pc,Sigiriya_pc,Nuwara Eliya_pc,Yala_pc,Hassan_pc,Hampi_pc,Nasik_pc,Saputara_pc,Cherrapunjee_pc,Mathura_pc,Thim

(9000, 416)
